In [1]:
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = "2"

import tensorflow as tf
import numpy as np
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.optimizers import Adam


In [24]:
tf.random.set_seed(22)
np.random.seed(22)

#assert tf.__version__.startswith('2.')

batch_size = 128
total_words = 10000
max_review_len = 80
embedding_len = 100


In [25]:
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.imdb.load_data(num_words=total_words)

In [26]:
print(x_train.shape)
print(y_train.shape)
print(x_test.shape)

(25000,)
(25000,)
(25000,)


In [27]:
print(x_train[0])

[1, 14, 22, 16, 43, 530, 973, 1622, 1385, 65, 458, 4468, 66, 3941, 4, 173, 36, 256, 5, 25, 100, 43, 838, 112, 50, 670, 2, 9, 35, 480, 284, 5, 150, 4, 172, 112, 167, 2, 336, 385, 39, 4, 172, 4536, 1111, 17, 546, 38, 13, 447, 4, 192, 50, 16, 6, 147, 2025, 19, 14, 22, 4, 1920, 4613, 469, 4, 22, 71, 87, 12, 16, 43, 530, 38, 76, 15, 13, 1247, 4, 22, 17, 515, 17, 12, 16, 626, 18, 2, 5, 62, 386, 12, 8, 316, 8, 106, 5, 4, 2223, 5244, 16, 480, 66, 3785, 33, 4, 130, 12, 16, 38, 619, 5, 25, 124, 51, 36, 135, 48, 25, 1415, 33, 6, 22, 12, 215, 28, 77, 52, 5, 14, 407, 16, 82, 2, 8, 4, 107, 117, 5952, 15, 256, 4, 2, 7, 3766, 5, 723, 36, 71, 43, 530, 476, 26, 400, 317, 46, 7, 4, 2, 1029, 13, 104, 88, 4, 381, 15, 297, 98, 32, 2071, 56, 26, 141, 6, 194, 7486, 18, 4, 226, 22, 21, 134, 476, 26, 480, 5, 144, 30, 5535, 18, 51, 36, 28, 224, 92, 25, 104, 4, 226, 65, 16, 38, 1334, 88, 12, 16, 283, 5, 16, 4472, 113, 103, 32, 15, 16, 5345, 19, 178, 32]


In [28]:
print(y_train[0])

1


In [29]:
x_train = tf.keras.preprocessing.sequence.pad_sequences(x_train, maxlen=max_review_len)
x_test = tf.keras.preprocessing.sequence.pad_sequences(x_test, maxlen=max_review_len)

In [30]:
print(x_train[0])

[  15  256    4    2    7 3766    5  723   36   71   43  530  476   26
  400  317   46    7    4    2 1029   13  104   88    4  381   15  297
   98   32 2071   56   26  141    6  194 7486   18    4  226   22   21
  134  476   26  480    5  144   30 5535   18   51   36   28  224   92
   25  104    4  226   65   16   38 1334   88   12   16  283    5   16
 4472  113  103   32   15   16 5345   19  178   32]


In [31]:
print(x_train[1])

[ 125   68    2 6853   15  349  165 4362   98    5    4  228    9   43
    2 1157   15  299  120    5  120  174   11  220  175  136   50    9
 4373  228 8255    5    2  656  245 2350    5    4 9837  131  152  491
   18    2   32 7464 1212   14    9    6  371   78   22  625   64 1382
    9    8  168  145   23    4 1690   15   16    4 1355    5   28    6
   52  154  462   33   89   78  285   16  145   95]


In [32]:
train_data = tf.data.Dataset.from_tensor_slices((x_train, y_train))
train_data = train_data.shuffle(10000).batch(batch_size, drop_remainder=True)
test_data = tf.data.Dataset.from_tensor_slices((x_train, y_train))
test_data = test_data.batch(batch_size, drop_remainder=True)

In [33]:
print(x_train.shape)

(25000, 80)


In [40]:
class RNN_Build(tf.keras.Model) :
    def __init__(self, units) :
        super(RNN_Build, self).__init__()
        self.embedding = tf.keras.layers.Embedding(total_words, 
                                                   embedding_len, 
                                                   input_length=max_review_len)
        self.rnn = tf.keras.Sequential([
            tf.keras.layers.GRU(units, dropout=0.2, return_sequences=True),
            tf.keras.layers.GRU(units, dropout=0.2)
        ])
        self.outlayer = tf.keras.layers.Dense(1)

    def call(self, inputs, training=None) :
        x = inputs
        x = self.embedding(x)
        x = self.rnn(x)
        x = self.outlayer(x)
        prob = tf.sigmoid(x)
        return prob
        

In [41]:
units = 64
epochs = 4
model = RNN_Build(units)
model.compile(optimizer=Adam(0.001),
             loss=tf.losses.BinaryCrossentropy(),
             metrics=['accuracy'],
             experimental_run_tf_function=False)


In [42]:
history = model.fit(train_data, epochs=epochs, validation_data=test_data)

Epoch 1/4


NotImplementedError: in user code:

    C:\Users\teacher\.conda\envs\tf2_book\lib\site-packages\tensorflow\python\keras\engine\training.py:806 train_function  *
        return step_function(self, iterator)
    C:\Users\teacher\AppData\Local\Temp\ipykernel_18396\931888230.py:16 call  *
        x = self.rnn(x)
    C:\Users\teacher\.conda\envs\tf2_book\lib\site-packages\tensorflow\python\keras\engine\base_layer.py:985 __call__  **
        outputs = call_fn(inputs, *args, **kwargs)
    C:\Users\teacher\.conda\envs\tf2_book\lib\site-packages\tensorflow\python\keras\engine\sequential.py:386 call
        outputs = layer(inputs, **kwargs)
    C:\Users\teacher\.conda\envs\tf2_book\lib\site-packages\tensorflow\python\keras\layers\recurrent.py:663 __call__
        return super(RNN, self).__call__(inputs, **kwargs)
    C:\Users\teacher\.conda\envs\tf2_book\lib\site-packages\tensorflow\python\keras\engine\base_layer.py:985 __call__
        outputs = call_fn(inputs, *args, **kwargs)
    C:\Users\tea